In [1]:
DL = '/Users/damiansp/Downloads'

In [2]:
!rm -rf $DL/*

In [3]:
from datetime import datetime, timedelta
import os

import numpy as np
import pandas as pd
import xlrd

In [4]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ACN', 'ADBE', 'ADYEY', 'AMD', 'AMGN',
    'AMZN', 'ANET', 'ASML', 'AXON', 'BROS', 'BYND', 'CHWY', 'CRWD',
    'DDOG', 'DIS', 'DOCS', 'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT',
    'FTNT', 'GH', 'GMED', 'GOOG', 'IDXX', 'INTG', 'ISRG', 'JD',
    'LRCX', 'LSPD', 'LULU', 'MA', 'MASI', 'MDB', 'MELI', 'MNST',
    'MSFT', 'MTCH', 'NET', 'NOW', 'NVCR', 'NVDA', 'ODFL', 'OKTA',
    'OPEN', 'PAC', 'PANW', 'PAYC', 'PINS', 'PTON', 'PYPL', 'QDEL',
    'RBLX', 'RDFN', 'RGEN', 'RIVN', 'RMD', 'ROKU', 'RVLV', 'SAM',
    'SBUX', 'SE', 'SHOP', 'SNOW', 'TASK', 'TDOC', 'TEAM', 'TMUS', 
    'TREX', 'TSCO', 'TSLA', 'TTD', 'TTWO', 'TWLO', 'TWTR', 'TYL',
    'UPST', 'VEEV', 'VLD', 'VRTX', 'XPEV', 'ZEN', 'ZM']
LINGERERS = [
    'AMEH', 'AMRK', 'BRTX', 'BTTR', 'BYRN', 'CENN', 'CGAU', 'CHEF',
    'CIZN', 'COOP', 'CTGO', 'CVE', 'DQ', 'DTST', 'DYAI', 'EP', 'FCUV',
    'FKWL', 'GGE', 'GME', 'HBM', 'HNRG', 'IDR', 'INTZ',  'KREF',
    'MCVT', 'MIY', 'MNMD', 'MOBQ', 'NAN', 'OMQS', 'PACB', 'PETV',
    'POWW', 'PRPH', 'RVP', 'RVYL', 'SOBR', 'VIRT', 'WAVD', 'WKHS',
    'ZEST', 'ZYXI']
BEST_WEIGHTED = [
    'ABST', 'AGI', 'APT', 'BCTF', 'CEF', 'CZFS', 'DGX', 'ESOA',
    'FCN', 'FNV', 'FSI', 'GCBC', 'HMY', 'LFMD', 'MICS', 'MRTI',
    'MTA', 'NEN', 'UTMD', 'YORW']

SYMBOLS = sorted(list(set(SYMBOLS + LINGERERS + BEST_WEIGHTED)))
len(SYMBOLS)

150

In [5]:
!ls $DL

In [6]:
#!head -5 $DL/Positions.csv

In [7]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [8]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [9]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ABST
ACN


### E*Trade

In [10]:
TODAY = datetime.now().date()
TODAY
DONGMEI = False

In [11]:
df = pd.read_csv(
    f'{DL}/Positions.csv', index_col=0, skiprows=1
)[['Market Value']].rename(columns={'Market Value': 'ET'})
df.index = map(lambda x: x.split()[0], df.index)
df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
df.head()

,ET
AAPL,2701.04
ABMD,2284.5
ABNB,189.4
ACN,2595.69
ADBE,2645.12


In [12]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    et_df.ET = et_df.ET.apply(
        lambda x: round(float(x)) if x != '--' else x)
et_df.head()

,ET
AAPL,2701
ABMD,2284
ABNB,189
ACN,2596
ADBE,2645


In [13]:
#et_df.drop(index=['98986CONT'], inplace=True)
et_df.head()

,ET
AAPL,2701
ABMD,2284
ABNB,189
ACN,2596
ADBE,2645


In [14]:
#et_df.drop('98936J101', inplace=True)

In [15]:
et_df.loc[et_df.ET == '--']
#et_df.loc['TASK', 'ET'] = 19.24

,ET


In [16]:
check_symbols(et_df)

All clear


In [17]:
#et_df = et_df[1:]

In [18]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,2701
ABMD,2284
ABNB,189
ABST,0
ACN,2596


### Fidelity

In [19]:
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Dec-09-2022.csv'

In [20]:
!ls $DL

Portfolio_Positions_Dec-09-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_12_09.xls


In [21]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("-05-", "-06-")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,Rollover IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,5878.24,$1.00,$0.00,$5878.24,NaN,NaN,NaN,NaN,16.50%,NaN,NaN,Cash
1,218520300,Rollover IRA,MCVT,MILL CITY VENTURES III LTD COM NEW,1.00,$2.23,-$0.42,$2.23,-$0.42,-15.85%,+$0.44,+24.58%,0.01%,$1.79,$1.79,Cash
2,218520300,Rollover IRA,LULU,LULULEMON ATHLETICA INC COM USD0.005,3.00,$326.39,-$48.12,$979.17,-$144.36,-12.85%,+$126.65,+14.85%,2.75%,$852.52,$284.17,Cash
3,218520300,Rollover IRA,GME,GAMESTOP CORPORATION COM USD0.001 CLASS A,3.00,$22.63,-$2.16,$67.89,-$6.48,-8.72%,-$8.10,-10.66%,0.19%,$75.99,$25.33,Cash
4,218520300,Rollover IRA,BYND,BEYOND MEAT INC COM,4.00,$14.87,-$1.28,$59.48,-$5.12,-7.93%,-$28.52,-32.41%,0.17%,$88.00,$22.00,Cash


In [22]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,Rollover IRA,MCVT,2
2,Rollover IRA,LULU,979
3,Rollover IRA,GME,68
4,Rollover IRA,BYND,59
5,Rollover IRA,EDIT,340


In [23]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['Rollover IRA'].head()

,Rollover IRA
Symbol,
MCVT,2
LULU,979
GME,68
BYND,59
EDIT,340


In [24]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

Rollover IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [25]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA
AAPL,2701,0,569,0
ABMD,2284,1523,0,0
ABNB,189,0,95,0
ABST,0,0,0,0
ACN,2596,0,1730,0


### TD Ameritrade

In [26]:
#!cat $DL/Positions_damiansp_Stocks_2022_08_09.xls

In [27]:
today = str(TODAY).replace('-', '_')
path = f'{DL}/Positions_damiansp_Stocks_{today}.xls'
#tomorrow = str(TODAY + timedelta(1)).replace('-', '_')
#path = f'{DL}/Positions_damiansp_Stocks_{tomorrow}.xls'

#path = f'{DL}/Positions_damiansp_Stocks_2022_11_04.xls'

In [28]:
# Stock (default) view
#td = pd.read_excel(, engine='xlrd')
def parse_file(path):
    inds = []
    data = []
    is_header = True
    with open(path, 'r') as f:
        for line in f:
            if not is_header:
                if line.startswith('"'):
                    try:
                        columns = line.split('\t')
                        symbol = columns[0][1:-1]
                        value = float(columns[5][1:-1].replace(',', ''))
                        inds.append(symbol)
                        data.append(value)
                    except IndexError:
                        print('Cannot parse:', line)
            if line.startswith('Symbol'):
                is_header = False
    return pd.DataFrame({'TD': data}, index=inds)

In [29]:
td = parse_file(path)
td.head()

,TD
ABST,37.32
AGI,78.72
AMEH,114.56
AMRK,245.14
AMZN,177.76


In [30]:
out = pd.concat([out, td], axis=1).fillna(0).round().astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA,TD
AAPL,2701,0,569,0,0
ABMD,2284,1523,0,0,0
ABNB,189,0,95,0,0
ABST,0,0,0,0,37
ACN,2596,0,1730,0,0


In [31]:
out.to_csv(f'{DL}/clean.csv')

In [32]:
!open $DL/clean.csv

In [31]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [10]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


In [11]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [12]:
check_symbols(mw)

All clear


In [13]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [14]:
mw.to_csv(f'{DL}/mw.csv')

In [47]:
!open $DL/mw.csv

The file /Users/damiansp/Downloads/mw.csv does not exist.


In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*